In [ ]:
# sorry for installs here, I'll move them to poetry later
! pip install -U jupyter ipywidgets # for tqdm to function properly
! pip install openpyxl # for saving stuff to Excel files

In [6]:
import sys
import json
import uuid
from tqdm.notebook import tqdm
import datetime 
from openpyxl import Workbook
import queue

# path is broken on my machine, so I leave this here for myself :)
sys.path.append('/Users/veronicasmilga/Desktop/Tübingen/MSE/Project_MSE/')

from db.DocumentEntry import DocumentEntry
from db.DocumentRepository import DocumentRepository
from data_retrieval.Crawler import Crawler

In [7]:
# frontier now is in a separate file
with open("../frontier.json", "r") as file:
    frontier = json.load(file)

NB: for the database to function properly, please first go to `exp/001_Flat_db_example_connection.ipynb` and complete the steps from there. If you don't want to be saving documents to the database, just comment out the code after _"# save one crawled page to database"_ comment.

In [8]:
# # initialising the database
# documentRepository = DocumentRepository()

# initialising the Excel backup (if sth goes wrong with the database)
wb = Workbook()
ws = wb.active
ws.title = "Crawled Data"
headers = ["id", "url", "title", "headings", "page_text", "keywords", "accessed_timestamp", "internal_links", "external_links"]
ws.append(headers)

In [ ]:
now = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

max_pages = 5
max_steps_per_domain = 3 # only applies to websites NOT about Tuebingen
timeout = 10

# Crawler is an iterator now, to handle info generated on-the-fly and save it immediately
crawler = Crawler(frontier, max_pages, max_steps_per_domain, timeout)

for scraped_webpage_info, to_visit_prioritised, to_visit, visited_domains, visited in tqdm(crawler, total=max_pages):
    # save one crawled page to excel file
    row = [
        str(uuid.uuid4()),
        scraped_webpage_info["url"],
        scraped_webpage_info["title"],
        str(scraped_webpage_info["headings"]),
        scraped_webpage_info["page_text"],
        str(scraped_webpage_info["keywords"]),
        scraped_webpage_info["accessed_timestamp"],
        str(scraped_webpage_info["internal_links"]),
        str(scraped_webpage_info["external_links"])
    ]
    ws.append(row)
    wb.save(f"./data/crawled_data_backup_{now}.xlsx")

    # # save one crawled page to database
    # document = DocumentEntry(
    #     url=scraped_webpage_info["url"],
    #     title=scraped_webpage_info["title"],
    #     headings=scraped_webpage_info["headings"],
    #     page_text=scraped_webpage_info["page_text"], 
    #     keywords=scraped_webpage_info["keywords"],
    #     accessed_timestamp=scraped_webpage_info["accessed_timestamp"],
    #     internal_links=scraped_webpage_info["internal_links"],
    #     external_links=scraped_webpage_info["external_links"],
    #     id=uuid.uuid4()
    #     )
    # documentRepository.saveDocument(document)

    crawling_state = {
        "to_visit_prioritised": list(to_visit_prioritised.queue), 
        "to_visit": list(to_visit.queue), 
        "visited_domains": list(visited_domains), 
        "visited": list(visited)
    }

    json_filename = f"data/current_state_backup_file_{now}.json"
    with open(json_filename, "w") as f:
        json.dump(crawling_state, f, indent=4)
    
    print(f"Saved checkpoint info to {json_filename}.")

In [ ]:
now = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

with open("data/current_state_backup_file_2024-07-04_21-27-38.json", "r") as f:
    crawling_state = json.load(f)

max_pages = 5
max_steps_per_domain = 3 # only applies to websites NOT about Tuebingen
timeout = 10

to_visit_list = crawling_state["to_visit"]
to_visit_prioritised_list = crawling_state["to_visit_prioritised"]
visited_list = crawling_state["visited"]
visited_domains_list = crawling_state["visited_domains"]

to_visit = queue.Queue()
for item in to_visit_list:
    to_visit.put(item)

to_visit_prioritised = queue.Queue()
for item in to_visit_prioritised_list:
    to_visit_prioritised.put(item)

visited = set(visited)
visited_domains = set(visited_domains)

# Crawler is an iterator now, to handle info generated on-the-fly and save it immediately
crawler = Crawler(
    frontier,
    max_pages, 
    max_steps_per_domain, 
    timeout,
    to_visit=to_visit,
    to_visit_prioritised=to_visit_prioritised,
    visited=visited,
    visited_domains=visited_domains,
    )

for scraped_webpage_info, to_visit_prioritised, to_visit, visited_domains, visited in tqdm(crawler, total=max_pages):
    # save one crawled page to excel file
    row = [
        str(uuid.uuid4()),
        scraped_webpage_info["url"],
        scraped_webpage_info["title"],
        str(scraped_webpage_info["headings"]),
        scraped_webpage_info["page_text"],
        str(scraped_webpage_info["keywords"]),
        scraped_webpage_info["accessed_timestamp"],
        str(scraped_webpage_info["internal_links"]),
        str(scraped_webpage_info["external_links"])
    ]
    ws.append(row)
    wb.save(f"./data/crawled_data_backup_{now}.xlsx")

    crawling_state = {
        "to_visit_prioritised": list(to_visit_prioritised.queue), 
        "to_visit": list(to_visit.queue), 
        "visited_domains": list(visited_domains), 
        "visited": list(visited)
    }

    json_filename = f"data/current_state_backup_file_{now}.json"
    with open(json_filename, "w") as f:
        json.dump(crawling_state, f, indent=4)
    
    print(f"Saved checkpoint info to {json_filename}.")

If you want to access documents in the database:

In [ ]:
# allDocuments = documentRepository.loadAllDocuments()

Otherwise, if you just want to take a look at the data, go to `exp/data` and find the Excel file with timestamp corresponding to the time you ran the Crawler :)